In [395]:
import numpy as np
import copy
import random

In [396]:
def net(w0, W, X):
    return (w0 + np.dot(W, X.T))[0]

def o(net):
    return 1 / (1 + np.e ** (-net))

def error(T, Z):
    return (0.5*(T[0]-Z[0])**2).sum()

def output_hidden_layers(X, HL):
    num_layers = HL.shape[0]
    print(f"Input: \n\t {X}")
    for i in range(1, num_layers + 1):
        if i < num_layers:
            print(f"Hidden layer #{i}: \n\t {HL[i-1]}")
            continue
        print(f"Output: \n\t {HL[i-1]}")

In [397]:
def forward_propagate(X, W, L, verbose=False):
    layers = L.copy()
    layers.append(X.shape[1])
    layers = np.array(layers)
    if layers.shape[0] != W.shape[0]:
        raise Exception("The length of W should be equal to the number of layers")
    R = []
    Yi = X
    for idx, neurons in enumerate(layers):
        W_matrix = W[idx]
        Y_temp = []
        for neuron in range(neurons):
            Wi = W_matrix[neuron]
            net_i = net(Wi[0], Wi[1:], Yi)
            Y_temp.append(o(net_i))
        Yi = np.array([Y_temp])
        R.append(Yi)
    if verbose:
        output_hidden_layers(X, np.array(R))
    return np.array(R)

In [398]:
X = np.array([[0.05, 0.1]])
T = np.array([[0.01, 0.99]])
hidden_layers = [2]
W = np.array([
        np.array([
            [0.35, 0.15, 0.2],
            [0.35, 0.25, 0.3]
        ]),
        np.array([
            [0.6, 0.4, 0.45],
            [0.6, 0.5, 0.55]
        ])
    ])
eta = 0.5

In [399]:
R = forward_propagate(X, W, hidden_layers, verbose=True)
Z = R[-1]
E = error(T, Z)
E

Input: 
	 [[0.05 0.1 ]]
Hidden layer #1: 
	 [[0.59326999 0.59688438]]
Output: 
	 [[0.75136507 0.77292847]]


0.2983711087600027

In [400]:
R

array([[[0.59326999, 0.59688438]],

       [[0.75136507, 0.77292847]]])

In [401]:
def backward_propagate(X, R, T, W, eta, hidden_layers, verbose=False):
    new_W = W[::-1]
    for idx, HL in enumerate(R[::-1]): #HL: hidden layer
        #Hidden Layers
        if idx < R.shape[0]-1:
            new_W[idx][:, 1:] = hidden_layer(HL, T, R, idx, new_W)
            continue
        initial_weights(X, HL, T, R, idx, new_W, eta)
    return new_W

def hidden_layer(HL, T, R, idx, W):
    d_EZ = HL - T
    d_Znet = HL*(1-HL)
    alpha = d_EZ * d_Znet
    d_netV = R[idx]
    d_EV = alpha.T @ d_netV
    return W[idx][:, 1:] - (eta * d_EV)

def initial_weights(X, HL, T, R, idx, new_W, eta):
    curr_W = new_W[1][:,1:]
    Y = R[0]
    Z = R[1]
    V = np.diagonal(new_W[0][:,1:])
    deltaE = np.sum(np.outer(V, (Z-T)*(Z*(1-Z))),axis=1)
    deltaEW = np.outer(deltaE*Y*(1-Y),X)
    new_weights = curr_W - eta*deltaEW
    new_W[1][:,1:] = new_weights

In [402]:
backward_propagate(X, R, T, W, eta, hidden_layers)

array([[[0.6       , 0.35891648, 0.40866619],
        [0.6       , 0.51130127, 0.56137012]],

       [[0.35      , 0.14978262, 0.19956523],
        [0.35      , 0.24966097, 0.29932193]]])